In [7]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import gridspec
import seaborn as sns
sns.set()

from sklearn.svm import SVC
from sklearn import svm, datasets
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

parameters = {}
KERNEL_LINEAR = 1
KERNEL_RBF = 2

DATASET_LINEARLY_SEPARABLE = 1
DATASET_CIRCULAR = 2

In [14]:
def get_data():
    data = np.loadtxt("iris.txt")
    iris_data = data[50:]
    return iris_data 

def get_train_and_test(feature):
    iris_data = get_data()

    positive_data = iris_data[:50, feature]
    positive_data = np.expand_dims(positive_data, axis=2)

    negative_data = iris_data[50:, feature]
    negative_data = np.expand_dims(negative_data, axis=2)

    train_data_positive = positive_data[:25]
    train_data_negative = negative_data[:25]

    test_data_positive = positive_data[25:]
    test_data_negative = negative_data[25:]

    train_data = np.concatenate((train_data_positive, train_data_negative), axis=0)
    test_data = np.concatenate((test_data_positive, test_data_negative), axis=0)

    train_label = [1.0] * len(train_data_positive) + [-1.0] * len(train_data_negative)
    test_label = [1.0] * len(test_data_positive) + [-1.0] * len(test_data_negative)

    train_label_plot = ['positive'] * len(train_data_positive) + ['negative'] * len(train_data_negative)
    test_label_plot = ['positive'] * len(test_data_positive) + ['negative'] * len(test_data_negative)

    return(train_data, test_data, train_label, test_label, train_label_plot, test_label_plot, train_data_positive, train_data_negative, test_data_positive, test_data_negative)

def get_test_and_train(feature):
    iris_data = get_data()

    positive_data = iris_data[:50, feature]
    positive_data = np.expand_dims(positive_data, axis=2)

    negative_data = iris_data[50:, feature]
    negative_data = np.expand_dims(negative_data, axis=2)

    train_data_positive = positive_data[25:]
    train_data_negative = negative_data[25:]

    test_data_positive = positive_data[:25]
    test_data_negative = negative_data[:25]

    train_data = np.concatenate((train_data_positive, train_data_negative), axis=0)
    test_data = np.concatenate((test_data_positive, test_data_negative), axis=0)

    train_label = [1.0] * len(train_data_positive) + [-1.0] * len(train_data_negative)
    test_label = [1.0] * len(test_data_positive) + [-1.0] * len(test_data_negative)

    train_label_plot = ['positive'] * len(train_data_positive) + ['negative'] * len(train_data_negative)
    test_label_plot = ['positive'] * len(test_data_positive) + ['negative'] * len(test_data_negative)

    return(train_data, test_data, train_label, test_label, train_label_plot, test_label_plot, train_data_positive, train_data_negative, test_data_positive, test_data_negative)

In [15]:
def generate_data1(dataset):
    feature = [2, 3]

    train_data, test_data, train_label, test_label, train_label_plot, test_label_plot, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_train_and_test(feature)

    train_data = np.squeeze(train_data, axis=(2,))
    train_label = np.array(train_label)

    X = train_data
    y = train_label 
    
    return X, y

def generate_data2(dataset):
    feature = [2, 3]

    train_data, test_data, train_label, test_label, train_label_plot, test_label_plot, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_test_and_train(feature)

    train_data = np.squeeze(train_data, axis=(2,))
    train_label = np.array(train_label)

    X = train_data
    y = train_label 
    
    return X, y

In [42]:
def gram_matrix(X, Y, kernel_type, gamma=200):
    K = np.zeros((X.shape[0], Y.shape[0]))
    
    if kernel_type == KERNEL_LINEAR:
        for i, x in enumerate(X):
            for j, y in enumerate(Y):
                K[i, j] = np.dot(x.T, y)
                
    elif kernel_type == KERNEL_RBF:
        for i, x in enumerate(X):
            for j, y in enumerate(Y):
                K[i, j] = np.exp(-gamma * np.linalg.norm(x - y) ** 2)
        
    return K

In [27]:
def train_svm1(kernel):
    C = 10

    feature = [2, 3]
    train_data, test_data, train_label, test_label, train_label_plot, test_label_plot, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_train_and_test(feature)

    train_data = np.squeeze(train_data, axis=(2,))
    train_label = np.array(train_label)

    n, k = train_data.shape
    
    y_matrix = y.reshape(1, -1)
    H = np.dot(y_matrix.T, y_matrix) * gram_matrix(X, X, kernel)
    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((n, 1)))
    G = cvxopt_matrix(np.vstack((-np.eye((n)), np.eye(n))))
    h = cvxopt_matrix(np.vstack((np.zeros((n,1)), np.ones((n,1)) * C)))
    A = cvxopt_matrix(y_matrix)
    b = cvxopt_matrix(np.zeros(1))
    
    cvxopt_solvers.options['abstol'] = 1e-10
    cvxopt_solvers.options['reltol'] = 1e-10
    cvxopt_solvers.options['feastol'] = 1e-10

    return cvxopt_solvers.qp(P, q, G, h, A, b)

def train_svm2(kernel):
    C = 10

    feature = [2, 3]
    train_data, test_data, train_label, test_label, train_label_plot, test_label_plot, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_test_and_train(feature)

    train_data = np.squeeze(train_data, axis=(2,))
    train_label = np.array(train_label)

    n, k = train_data.shape
    
    y_matrix = y.reshape(1, -1)
    H = np.dot(y_matrix.T, y_matrix) * gram_matrix(X, X, kernel)
    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((n, 1)))
    G = cvxopt_matrix(np.vstack((-np.eye((n)), np.eye(n))))
    h = cvxopt_matrix(np.vstack((np.zeros((n,1)), np.ones((n,1)) * C)))
    A = cvxopt_matrix(y_matrix)
    b = cvxopt_matrix(np.zeros(1))
    
    cvxopt_solvers.options['abstol'] = 1e-10
    cvxopt_solvers.options['reltol'] = 1e-10
    cvxopt_solvers.options['feastol'] = 1e-10

    return cvxopt_solvers.qp(P, q, G, h, A, b)

def train_svm3(kernel):
    C = 100

    feature = [2, 3]
    train_data, test_data, train_label, test_label, train_label_plot, test_label_plot, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_train_and_test(feature)

    train_data = np.squeeze(train_data, axis=(2,))
    train_label = np.array(train_label)

    n, k = train_data.shape
    
    y_matrix = y.reshape(1, -1)
    H = np.dot(y_matrix.T, y_matrix) * gram_matrix(X, X, kernel)
    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((n, 1)))
    G = cvxopt_matrix(np.vstack((-np.eye((n)), np.eye(n))))
    h = cvxopt_matrix(np.vstack((np.zeros((n,1)), np.ones((n,1)) * C)))
    A = cvxopt_matrix(y_matrix)
    b = cvxopt_matrix(np.zeros(1))
    
    cvxopt_solvers.options['abstol'] = 1e-10
    cvxopt_solvers.options['reltol'] = 1e-10
    cvxopt_solvers.options['feastol'] = 1e-10

    return cvxopt_solvers.qp(P, q, G, h, A, b)

def train_svm4(kernel):
    C = 100

    feature = [2, 3]
    train_data, test_data, train_label, test_label, train_label_plot, test_label_plot, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_test_and_train(feature)

    train_data = np.squeeze(train_data, axis=(2,))
    train_label = np.array(train_label)

    n, k = train_data.shape
    
    y_matrix = y.reshape(1, -1)
    H = np.dot(y_matrix.T, y_matrix) * gram_matrix(X, X, kernel)
    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((n, 1)))
    G = cvxopt_matrix(np.vstack((-np.eye((n)), np.eye(n))))
    h = cvxopt_matrix(np.vstack((np.zeros((n,1)), np.ones((n,1)) * C)))
    A = cvxopt_matrix(y_matrix)
    b = cvxopt_matrix(np.zeros(1))
    
    cvxopt_solvers.options['abstol'] = 1e-10
    cvxopt_solvers.options['reltol'] = 1e-10
    cvxopt_solvers.options['feastol'] = 1e-10

    return cvxopt_solvers.qp(P, q, G, h, A, b)

In [8]:
X, y = generate_data1(DATASET_LINEARLY_SEPARABLE)
# X = parameters['X']
svm_parameters = train_svm1(KERNEL_LINEAR)
print(svm_parameters)

pcost       dcost       gap    pres   dres
 0: -2.3310e+01 -2.9472e+03  6e+03  6e-01  2e-13
 1:  3.2470e+01 -7.2125e+02  1e+03  5e-02  2e-13
 2: -2.1288e+01 -1.6603e+02  2e+02  6e-03  1e-13
 3: -4.9203e+01 -9.4768e+01  5e+01  2e-03  2e-13
 4: -5.8984e+01 -7.2923e+01  1e+01  2e-04  2e-13
 5: -6.2339e+01 -7.0672e+01  8e+00  1e-04  2e-13
 6: -6.5178e+01 -6.6910e+01  2e+00  2e-05  2e-13
 7: -6.5868e+01 -6.5934e+01  7e-02  3e-08  2e-13
 8: -6.5900e+01 -6.5900e+01  7e-04  3e-10  2e-13
 9: -6.5900e+01 -6.5900e+01  7e-06  3e-12  1e-13
10: -6.5900e+01 -6.5900e+01  7e-08  3e-14  2e-13
11: -6.5900e+01 -6.5900e+01  7e-10  1e-14  2e-13
Optimal solution found.
{'x': <50x1 matrix, tc='d'>, 'y': <1x1 matrix, tc='d'>, 's': <100x1 matrix, tc='d'>, 'z': <100x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 6.722454823557572e-10, 'relative gap': 1.0200993662505066e-11, 'primal objective': -65.8999999996739, 'dual objective': -65.9000000003458, 'primal infeasibility': 1.0658141036401503e-14, 'dual infeasibil

In [9]:
def get_parameters(alphas):
    threshold = 1e-5 # Values greater than zero (some floating point tolerance)
    S = (alphas > threshold).reshape(-1, )
    w = np.dot(X.T, alphas * y)
    b = y[S] - np.dot(X[S], w) # b calculation
    b = np.mean(b)
    return w, b, S

alphas = np.array(svm_parameters['x'])[:, 0]
w, b, S = get_parameters(alphas)

print('Alphas:', alphas[S][0:50])
print('w and b', w, b)

Alphas: [ 9.00006523 10.         10.          8.99993477  8.         10.
 10.         10.        ]
w and b [-1.6 -4.2] 14.74250000013928


In [10]:
# alphas
for i in range(len(alphas)):
    print('{}'.format(round(alphas[i], 4)))

0.0
0.0
9.0001
0.0
0.0
0.0
10.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
10.0
0.0
8.9999
0.0
0.0
0.0
8.0
0.0
0.0
0.0
0.0
10.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
10.0
0.0
0.0
0.0
10.0
0.0


In [16]:
X, y = generate_data2(DATASET_LINEARLY_SEPARABLE)
# X = parameters['X']
svm_parameters = train_svm2(KERNEL_LINEAR)
print(svm_parameters)

pcost       dcost       gap    pres   dres
 0: -2.6633e+01 -3.1443e+03  7e+03  6e-01  2e-13
 1:  4.5308e+01 -6.8677e+02  8e+02  2e-02  2e-13
 2: -2.4238e+01 -1.3871e+02  1e+02  2e-03  2e-13
 3: -4.7890e+01 -7.9039e+01  3e+01  4e-04  2e-13
 4: -5.4621e+01 -7.4160e+01  2e+01  2e-04  2e-13
 5: -6.0786e+01 -6.6426e+01  6e+00  2e-05  3e-13
 6: -6.2727e+01 -6.3719e+01  1e+00  4e-06  2e-13
 7: -6.3102e+01 -6.3127e+01  2e-02  4e-08  3e-13
 8: -6.3111e+01 -6.3111e+01  4e-04  4e-10  3e-13
 9: -6.3111e+01 -6.3111e+01  4e-06  4e-12  3e-13
10: -6.3111e+01 -6.3111e+01  4e-08  4e-14  3e-13
11: -6.3111e+01 -6.3111e+01  4e-10  7e-15  3e-13
Optimal solution found.
{'x': <50x1 matrix, tc='d'>, 'y': <1x1 matrix, tc='d'>, 's': <100x1 matrix, tc='d'>, 'z': <100x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 3.5834624662963347e-10, 'relative gap': 5.6780215135083095e-12, 'primal objective': -63.111111110993335, 'dual objective': -63.11111111135118, 'primal infeasibility': 7.105427357601002e-15, 'dual infeasi

In [18]:
def get_parameters(alphas):
    threshold = 1e-5 # Values greater than zero (some floating point tolerance)
    S = (alphas > threshold).reshape(-1, )
    w = np.dot(X.T, alphas * y)
    b = y[S] - np.dot(X[S], w) # b calculation
    b = np.mean(b)
    return w, b, S

alphas = np.array(svm_parameters['x'])[:, 0]
w, b, S = get_parameters(alphas)

print('Alphas:', alphas[S][0:50])
print('w and b', w, b)

Alphas: [10.         10.          8.77777778 10.         10.          8.55555555
 10.          0.22222222 10.        ]
w and b [-2.66666667 -4.66666667] 20.696296296583967


In [19]:
# alphas
for i in range(len(alphas)):
    print('{}'.format(round(alphas[i], 4)))

0.0
0.0
10.0
0.0
0.0
0.0
0.0
0.0
10.0
0.0
8.7778
10.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
10.0
8.5556
0.0
0.0
0.0
0.0
0.0
10.0
0.2222
0.0
0.0
0.0
10.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [21]:
X, y = generate_data1(DATASET_LINEARLY_SEPARABLE)
# X = parameters['X']
svm_parameters = train_svm3(KERNEL_LINEAR)
print(svm_parameters)

def get_parameters(alphas):
    threshold = 1e-5 # Values greater than zero (some floating point tolerance)
    S = (alphas > threshold).reshape(-1, )
    w = np.dot(X.T, alphas * y)
    b = y[S] - np.dot(X[S], w) # b calculation
    b = np.mean(b)
    return w, b, S

alphas = np.array(svm_parameters['x'])[:, 0]
w, b, S = get_parameters(alphas)

print('Alphas:', alphas[S][0:50])
print('w and b', w, b)

# alphas
for i in range(len(alphas)):
    print('{}'.format(round(alphas[i], 4)))

pcost       dcost       gap    pres   dres
 0:  5.0361e+03 -2.2918e+05  4e+05  4e-01  2e-12
 1:  7.5074e+03 -4.2773e+04  6e+04  3e-02  2e-12
 2:  1.7487e+03 -7.4395e+03  1e+04  3e-03  1e-12
 3: -1.5499e+02 -1.4054e+03  1e+03  5e-05  1e-12
 4: -3.5559e+02 -1.0313e+03  7e+02  2e-05  9e-13
 5: -4.4018e+02 -6.3139e+02  2e+02  6e-06  1e-12
 6: -5.2379e+02 -6.5827e+02  1e+02  2e-06  1e-12
 7: -5.1956e+02 -6.0815e+02  9e+01  7e-14  1e-12
 8: -5.5485e+02 -5.5807e+02  3e+00  3e-14  1e-12
 9: -5.5553e+02 -5.5563e+02  1e-01  7e-14  2e-12
10: -5.5555e+02 -5.5556e+02  5e-03  4e-14  2e-12
11: -5.5556e+02 -5.5556e+02  5e-04  9e-14  2e-12
12: -5.5556e+02 -5.5556e+02  7e-05  2e-13  2e-12
13: -5.5556e+02 -5.5556e+02  1e-05  3e-14  2e-12
14: -5.5556e+02 -5.5556e+02  1e-06  9e-14  2e-12
15: -5.5556e+02 -5.5556e+02  2e-07  3e-14  1e-12
16: -5.5556e+02 -5.5556e+02  3e-08  1e-14  2e-12
Optimal solution found.
{'x': <50x1 matrix, tc='d'>, 'y': <1x1 matrix, tc='d'>, 's': <100x1 matrix, tc='d'>, 'z': <100x1 mat

In [22]:
X, y = generate_data2(DATASET_LINEARLY_SEPARABLE)
# X = parameters['X']
svm_parameters = train_svm4(KERNEL_LINEAR)
print(svm_parameters)

def get_parameters(alphas):
    threshold = 1e-5 # Values greater than zero (some floating point tolerance)
    S = (alphas > threshold).reshape(-1, )
    w = np.dot(X.T, alphas * y)
    b = y[S] - np.dot(X[S], w) # b calculation
    b = np.mean(b)
    return w, b, S

alphas = np.array(svm_parameters['x'])[:, 0]
w, b, S = get_parameters(alphas)

print('Alphas:', alphas[S][0:50])
print('w and b', w, b)

# alphas
for i in range(len(alphas)):
    print('{}'.format(round(alphas[i], 4)))

pcost       dcost       gap    pres   dres
 0:  4.4172e+03 -2.4733e+05  5e+05  5e-01  2e-12
 1:  8.5358e+03 -3.3820e+04  4e+04  2e-03  2e-12
 2:  1.1755e+03 -4.1302e+03  5e+03  1e-04  2e-12
 3: -2.2616e+02 -1.3069e+03  1e+03  3e-14  1e-12
 4: -3.8397e+02 -6.6448e+02  3e+02  1e-14  1e-12
 5: -4.3299e+02 -5.1997e+02  9e+01  7e-14  1e-12
 6: -4.5523e+02 -5.1389e+02  6e+01  1e-16  1e-12
 7: -4.7717e+02 -4.7871e+02  2e+00  7e-14  2e-12
 8: -4.7755e+02 -4.7756e+02  2e-02  1e-13  2e-12
 9: -4.7755e+02 -4.7755e+02  2e-04  9e-14  2e-12
10: -4.7755e+02 -4.7755e+02  2e-06  4e-14  2e-12
11: -4.7755e+02 -4.7755e+02  2e-08  1e-13  2e-12
Optimal solution found.
{'x': <50x1 matrix, tc='d'>, 'y': <1x1 matrix, tc='d'>, 's': <100x1 matrix, tc='d'>, 'z': <100x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 1.639873344479713e-08, 'relative gap': 3.433922815392128e-11, 'primal objective': -477.5510204041822, 'dual objective': -477.55102042058394, 'primal infeasibility': 1.2789769243681803e-13, 'dual infeasib

In [43]:
X, y = generate_data1(DATASET_LINEARLY_SEPARABLE)
# X = parameters['X']
svm_parameters = train_svm1(KERNEL_RBF)
print(svm_parameters)

def get_parameters(alphas):
    threshold = 1e-5 # Values greater than zero (some floating point tolerance)
    S = (alphas > threshold).reshape(-1, )
    w = np.dot(X.T, alphas * y)
    b = y[S] - np.dot(X[S], w) # b calculation
    b = np.mean(b)
    return w, b, S

alphas = np.array(svm_parameters['x'])[:, 0]
w, b, S = get_parameters(alphas)

print('Alphas:', alphas[S][0:50])
print('w and b', w, b)

# alphas
for i in range(len(alphas)):
    print('{}'.format(round(alphas[i], 4)))
    if((i+1)%10==0):
        print("----")

pcost       dcost       gap    pres   dres
 0:  1.7770e+02 -1.4416e+03  2e+03  4e-15  2e-15
 1:  2.2010e+01 -1.5362e+02  2e+02  2e-15  1e-15
 2: -1.8929e+01 -3.9571e+01  2e+01  4e-16  4e-16
 3: -2.1360e+01 -2.2532e+01  1e+00  4e-15  2e-16
 4: -2.1377e+01 -2.1392e+01  1e-02  7e-16  1e-16
 5: -2.1377e+01 -2.1377e+01  1e-04  2e-15  1e-16
 6: -2.1377e+01 -2.1377e+01  1e-06  4e-15  1e-16
 7: -2.1377e+01 -2.1377e+01  1e-08  2e-16  1e-16
 8: -2.1377e+01 -2.1377e+01  1e-10  9e-16  1e-16
Optimal solution found.
{'x': <50x1 matrix, tc='d'>, 'y': <1x1 matrix, tc='d'>, 's': <100x1 matrix, tc='d'>, 'z': <100x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 1.4694045510154807e-10, 'relative gap': 6.873688062938697e-12, 'primal objective': -21.377236464048508, 'dual objective': -21.377236464195448, 'primal infeasibility': 8.881784197001252e-16, 'dual infeasibility': 1.0855906073093382e-16, 'primal slack': 0.321302058731487, 'dual slack': 1.0470498258845193e-14, 'iterations': 8}
Alphas: [0.53765599 0.32

In [44]:
X, y = generate_data2(DATASET_LINEARLY_SEPARABLE)
# X = parameters['X']
svm_parameters = train_svm2(KERNEL_RBF)
print(svm_parameters)

def get_parameters(alphas):
    threshold = 1e-5 # Values greater than zero (some floating point tolerance)
    S = (alphas > threshold).reshape(-1, )
    w = np.dot(X.T, alphas * y)
    b = y[S] - np.dot(X[S], w) # b calculation
    b = np.mean(b)
    return w, b, S

alphas = np.array(svm_parameters['x'])[:, 0]
w, b, S = get_parameters(alphas)

print('Alphas:', alphas[S][0:50])
print('w and b', w, b)

# alphas
for i in range(len(alphas)):
    print('{}'.format(round(alphas[i], 4)))
    if((i+1)%10==0):
        print("----")

pcost       dcost       gap    pres   dres
 0:  1.8277e+02 -1.3193e+03  2e+03  2e-14  2e-15
 1:  2.0491e+01 -1.4269e+02  2e+02  2e-14  1e-15
 2: -1.8414e+01 -3.7875e+01  2e+01  7e-15  5e-16
 3: -2.0692e+01 -2.1783e+01  1e+00  1e-15  2e-16
 4: -2.0710e+01 -2.0724e+01  1e-02  9e-16  2e-16
 5: -2.0710e+01 -2.0710e+01  1e-04  9e-16  2e-16
 6: -2.0710e+01 -2.0710e+01  1e-06  3e-15  1e-16
 7: -2.0710e+01 -2.0710e+01  1e-08  3e-15  1e-16
 8: -2.0710e+01 -2.0710e+01  1e-10  9e-16  1e-16
Optimal solution found.
{'x': <50x1 matrix, tc='d'>, 'y': <1x1 matrix, tc='d'>, 's': <100x1 matrix, tc='d'>, 'z': <100x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 1.418942531078335e-10, 'relative gap': 6.8514308634677425e-12, 'primal objective': -20.71016345861746, 'dual objective': -20.710163458759354, 'primal infeasibility': 8.881784197001252e-16, 'dual infeasibility': 1.1118780802721762e-16, 'primal slack': 0.3535428924892867, 'dual slack': 9.306676991717254e-15, 'iterations': 8}
Alphas: [0.92828708 1.034